Basado en: 
https://medium.com/@gayani.parameswaran/q-a-chatbot-using-langchain-hugging-face-and-mistral-31cd576779ef

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEndpoint
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
import yaml

c:\Users\feder\anaconda3\envs\nlp\Lib\importlib\__init__.py:126: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)


In [3]:
with open("keys.yaml") as keys:
    try:
        api_keys = yaml.safe_load(keys)
    except yaml.YAMLError as exc:
        print(exc)

In [4]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = api_keys["hf_model"]

In [19]:
## Read the pdfs from the folder
loader=PyPDFDirectoryLoader("./data/wikipedia")
documents=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=5000,chunk_overlap=200)
final_documents=text_splitter.split_documents(documents)
len(final_documents) #length --> 316

1626

In [20]:
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="sentence-transformers/all-MiniLM-l6-v2",
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}
)

vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [23]:
query="What is AES Corporation?"
relevant_docments=vectorstore.similarity_search(query)
print(relevant_docments[0].page_content)

■The AES Corporation is an American utility and power generation company. It owns and operates power plants, which it uses to generate and sell electricity to end users and intermediaries like utilities and industrial facilities. AES, headquartered in Arlington, Virginia, is one of the world's leading power companies, generating and distributing electric power in 15 countries[2] and employing 10,500 people worldwide. AES Corporation is a global Fortune 500 power company.[3] AES Ranks in the Top Ten of Fast Company's 2022 Best Workplaces for Innovators.[4]■The company was founded on January 28, 1981, as Applied Energy Services[5] by Roger Sant and Dennis Bakke, two appointees of the Federal Energy Administration under president Richard Nixon. The company was initially a consulting firm; it became AES Corporation, which went public in 1991. Sant was chairman, CEO, and president and Bakke was executive vice president until assuming the position of president in 1987. Bakke later became the

In [8]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [12]:
hf=HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-v0.3",
    temperature=0.1,
    max_length=500,
    huggingfacehub_api_token = os.environ["HUGGINGFACEHUB_API_TOKEN"]
)
query="What is the health insurance coverage?"
hf.invoke(query)

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\feder\.cache\huggingface\token
Login successful


HfHubHTTPError:  (Request ID: JiuZvGG7P4Ek00cekZlbQ)

403 Forbidden: None.
Cannot access content at: https://api-inference.huggingface.co/models/mistralai/Mistral-7B-v0.3.
If you are trying to create or update content, make sure you have a token with the `write` role.
The model mistralai/Mistral-7B-v0.3 is too large to be loaded automatically (14GB > 10GB). Please use Spaces (https://huggingface.co/spaces) or Inference Endpoints (https://huggingface.co/inference-endpoints).

In [8]:
from langchain_community.utilities import SQLDatabase
import os
import pandas as pd
import yaml
import sqlite3
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

In [9]:
conn = sqlite3.connect("data/database.db")
tickers = pd.read_sql_query("SELECT distinct Security FROM master_ticker", conn)["Security"].tolist()
db = SQLDatabase.from_uri("sqlite:///data/database.db")
print(db.get_usable_table_names())

['income_statements', 'master_ticker', 'price_data']


In [12]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = api_keys["hf_model"]

llm = ChatHuggingFace(model = "HuggingFaceH4/zephyr-7b-beta")

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "Which is the latest gross revenue of ACN?"})
response


ValidationError: 1 validation error for ChatHuggingFace
__root__
  Expected llm to be one of HuggingFaceTextGenInference, HuggingFaceEndpoint, HuggingFaceHub, HuggingFacePipeline received <class 'NoneType'> (type=type_error)

: 